# Content Based Recommender

In [199]:
import pandas as pd
import numpy as np
from pyvi import ViTokenizer
import glob
from collections import Counter
from string import punctuation

#Import data from the clean file 
product_df = pd.read_csv('product_info_clean.csv', index_col = 0)

#Print the head of the cleaned DataFrame
product_df.head()

,id,name,review_count,rating_average,price,description,category,category_name
0,21299202,Cáp sạc và truyền dữ liệu tốc độ cao Baseus Ho...,216,4.5,67000,"Hỗ trợ dòng lên đến 2.4 A , cho tốc độ sạc nha...",1815,Thiết Bị Số - Phụ Kiện Số/Phụ Kiện Điện Thoại ...
1,44105732,"Dép sục unisex duet độn đế, chống trượt Litic ...",27,3.5,148500,"Tiện lợi, bền đẹp, siêu nhẹ Độn đế tinh tế, kh...",914,Giày - Dép nữ/Giày lười nữ/Giày lười hở gót
2,54132740,bình thông hơi dành cho xe máy tặng bao tay xe...,4,4.5,199000,bình thông hơi dành cho xe máy tặng bao tay xe...,6061,Ô Tô - Xe Máy - Xe Đạp/Phụ kiện - Chăm sóc xe/...
3,65011722,"Đồng Hồ Nam FNGEEN FE230 mặt đồng hồ tròn, th...",19,4.5,179000,"Mặt đồng hồ tròn, thiết kế đẹp mắt, sáng bóng ...",914,Đồng hồ và Trang sức/Đồng hồ nam/Đồng hồ busin...
4,7995403,Giày Sneaker Unisex Converse Chuck Taylor All ...,19,5.0,1500000,"Đường may vô cùng tỉ mỉ, tinh tế Chất vải canv...",49620,Giày - Dép nam/Giày thể thao nam/Giày thể thao...


In [5]:
stop_word = []
with open("stop_word.txt",encoding="utf-8") as f:
    text = f.read()
    for word in text.split() :
        stop_word.append(word)
    f.close()
punc = list(punctuation)
stop_word = stop_word + punc
print(stop_word)

['bị', 'bởi', 'cả', 'các', 'cái', 'cần', 'càng', 'chỉ', 'chiếc', 'cho', 'chứ', 'chưa', 'chuyện', 'có', 'có_thể', 'cứ', 'của', 'cùng', 'cũng', 'đã', 'đang', 'đây', 'để', 'đến_nỗi', 'đều', 'điều', 'do', 'đó', 'được', 'dưới', 'gì', 'khi', 'không', 'là', 'lại', 'lên', 'lúc', 'mà', 'mỗi', 'một_cách', 'này', 'nên', 'nếu', 'ngay', 'nhiều', 'như', 'nhưng', 'những', 'nơi', 'nữa', 'phải', 'qua', 'ra', 'rằng', 'rằng', 'rất', 'rất', 'rồi', 'sau', 'sẽ', 'so', 'sự', 'tại', 'theo', 'thì', 'trên', 'trước', 'từ', 'từng', 'và', 'vẫn', 'vào', 'vậy', 'vì', 'việc', 'với', 'vừa', '!', '"', '#', '$', '%', '&', "'", ',', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\\\', ']', '^', '_', '`', '{', '|', '}', '~', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


In [6]:
def clean_text(text):
    text_lower = text.lower()
    text_token = ViTokenizer.tokenize(text_lower)
    
    sent = []
    for word in text_token.split("_\s") :
        if (word not in stop_word) :
            if ("_" in word) or (word.isalpha() == True):
                sent.append(word)
    return " ".join(sent)

product_df['description_token'] = product_df['description'].apply(clean_text)
product_df.head()

,id,name,review_count,rating_average,price,description,category,category_name,description_token
0,21299202,Cáp sạc và truyền dữ liệu tốc độ cao Baseus Ho...,216,4.5,67000,"Hỗ trợ dòng lên đến 2.4 A , cho tốc độ sạc nha...",1815,Thiết Bị Số - Phụ Kiện Số/Phụ Kiện Điện Thoại ...,"hỗ_trợ dòng lên đến 2.4 a , cho tốc_độ sạc nha..."
1,44105732,"Dép sục unisex duet độn đế, chống trượt Litic ...",27,3.5,148500,"Tiện lợi, bền đẹp, siêu nhẹ Độn đế tinh tế, kh...",914,Giày - Dép nữ/Giày lười nữ/Giày lười hở gót,"tiện_lợi , bền đẹp , siêu nhẹ độn đế tinh_tế ,..."
2,54132740,bình thông hơi dành cho xe máy tặng bao tay xe...,4,4.5,199000,bình thông hơi dành cho xe máy tặng bao tay xe...,6061,Ô Tô - Xe Máy - Xe Đạp/Phụ kiện - Chăm sóc xe/...,bình thông hơi dành cho xe_máy tặng bao_tay xe...
3,65011722,"Đồng Hồ Nam FNGEEN FE230 mặt đồng hồ tròn, th...",19,4.5,179000,"Mặt đồng hồ tròn, thiết kế đẹp mắt, sáng bóng ...",914,Đồng hồ và Trang sức/Đồng hồ nam/Đồng hồ busin...,"mặt đồng_hồ tròn , thiết_kế đẹp_mắt , sáng bón..."
4,7995403,Giày Sneaker Unisex Converse Chuck Taylor All ...,19,5.0,1500000,"Đường may vô cùng tỉ mỉ, tinh tế Chất vải canv...",49620,Giày - Dép nam/Giày thể thao nam/Giày thể thao...,"đường may vô_cùng tỉ_mỉ , tinh_tế chất vải can..."


In [7]:
product_df['name'].iloc[3]

'Đồng Hồ Nam FNGEEN FE230  mặt đồng hồ tròn, thiết kế đẹp mắt, sáng bóng với tính năng hiện đại cho phái mạnh tự tin, mạng mẽ và thời trang  Dây thép không gỉ thiết kế ôm tay '

In [8]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer(min_df=5,max_df= 0.8,max_features=3000,sublinear_tf=True)

#Replace NaN with an empty string
product_df['description_token'] = product_df['description_token'].fillna('')

#Construct the required TF-IDF matrix by applying the fit_transform method on the description_token feature
tfidf_matrix = tfidf.fit_transform(product_df['description_token'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(21838, 3000)

In [9]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [10]:
#Construct a reverse mapping of indices and product ids, and drop duplicate ids, if any
indices = pd.Series(product_df.index, index=product_df['id']).drop_duplicates()

In [11]:
# Function that takes in product id as input and gives recommendations 
def content_recommender(id, cosine_sim=cosine_sim, product_df=product_df, indices=indices):
    # Obtain the index of the product that matches the title
    idx = indices[id]

    # Get the pairwsie similarity scores of all products with that product
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the products based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 20 most similar products. Ignore the first product.
    sim_scores = sim_scores[1:21]

    # Get the product indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top 20 most similar products
    return product_df['name'].iloc[product_indices]

### Get recommendations for 
Đồng Hồ Nam FNGEEN FE230  mặt đồng hồ tròn, thiết kế đẹp mắt, sáng bóng với tính năng hiện đại cho phái mạnh tự tin, mạng mẽ và thời trang  Dây thép không gỉ thiết kế ôm tay 

In [12]:
donghonam_id = product_df[product_df['name'].str.find('Đồng Hồ Nam FNGEEN FE230  mặt đồng hồ tròn, thiết kế đẹp mắt, sáng bóng với tính năng hiện đại cho phái mạnh tự tin, mạng mẽ và thời trang  Dây thép không gỉ thiết kế ôm tay ') == 0]['id'].iloc[0]
content_recommender(donghonam_id)

7249     Đồng Hồ Nam FNGEEN FE708 mạ vàng mặt đồng hồ t...
11967    Đồng Hồ Nam FNGEEN FG0541 doanh nhân 2020 Lịch...
17822    Đồng hồ nam FNGEEN 5055 lịch ngày chính hãng n...
17906    Đồng Hồ Nam FNGEEN F281 mặt số đính đá Lịch Ng...
3290     Đồng hồ nam dây da thời trang - đồng hồ thể th...
17081    Đồng hồ nam ECONOMICXI dây thép mành đen chạy ...
13636    Đồng Hồ Nam Fourron F66 Lịch Ngày Dây Thép Cao...
15295    Đồng hồ nam chống nước NO6216G, phong cách trẻ...
2586     Đồng Hồ Nam Dây Thép Cao Cấp Thời Trang Lịch L...
477      Đồng hồ nam FNGEEN fn668 Mặt rồng 3d cuộn hình...
4799     Đồng Hồ Nam Fourron Lịch Ngày chính hãng nam s...
4918     Đồng Hồ Nam 2 Lịch Crnaira CR049 Dây Thép Mành...
6188     Đồng Hồ Nam Crnaira CR8333 Siêu Mỏng mặt đồng ...
10334    Đồng hồ Nam CUENA CUN8207 Lịch ngày sang trọng...
9980     Đồng hồ nam HALEI 564 mặt vuông thời trang cao...
9049     Đồng Hồ Nam Chạy 6 Kim Crnaira PE90 Mặt Đính Đ...
5144     Đồng Hồ Nam 6 Kim Crnaira C099 Dây Hợp Kim Thé.

In [13]:
import pickle
def save_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print("Save file: ", filename)
save_pickle(cosine_sim, 'cosine_sim.pkl')

Save file:  cosine_sim.pkl


# Collaborative Filtering

## The Framework

In [188]:
import pandas as pd
import numpy as np

In [3]:
#Load the review file into a dataframe
orig_ratings = pd.read_csv('product_review_clean.csv', index_col = 0)
orig_ratings.head()

/home/ktgiahieu/env/ds/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,customer_id,product_id,rating,thank_count,title,content,created_at,purchase_at,customer_name
0,5719606,21299202,5,6,"Dây 2m, chất liệu của dây dù",Xài được khoảng 1 tuần thì có vẻ ổn. Dây dài 2...,2020-08-25 14:04:15,2020-07-01 13:22:12,Khách Hàng
1,11891537,21299202,1,19,PSK Dragon giao láo bố láo.,"Đặt 3 sợi màu đỏ dài 2m , giao 2 sợi màu đen 2...",2020-08-12 11:41:41,2020-08-06 22:05:10,Trung Tung Tăng
2,7630980,21299202,4,0,Được,"cảm nhận ban đầu thì dây rất cứng, \ndây đúng ...",2020-09-04 11:15:21,2020-09-02 21:39:54,Nguyễn Tất Thắng
3,1242105,21299202,1,0,Rất không hài lòng,"Chất lượng hoàn thiện kém, kích thước chân lig...",2020-08-15 14:19:43,2020-08-13 20:50:11,Doan Hiep
4,10815590,21299202,4,8,Ổn,Hoàn thiện khá tốt\nNhưng tại sao dùng bút thử...,2020-03-08 13:23:01,2020-02-21 16:38:52,Hoàng việt hưng


## Graph approach

In [85]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(orig_ratings['customer_id'].apply(lambda x: 'u'+str(x)).unique(), bipartite = 'user')
G.add_nodes_from(orig_ratings['product_id'].apply(lambda x: 'i'+str(x)).unique(), bipartite = 'item')
product_customer_edges = [('u'+str(x['customer_id']), 'i'+str(x['product_id']), x['rating']) for i,x in orig_ratings.iterrows()]
G.add_weighted_edges_from(product_customer_edges)

### User-based collaborative filtering

In [286]:
user_nodes = [n for n in G.nodes() if G.nodes[n]['bipartite'] == 'user']
item_nodes = [n for n in G.nodes() if G.nodes[n]['bipartite'] == 'item']

In [297]:
def user_collaborative_recommender(user_id, similarity_metrics):
    user_u = 'u' + str(user_id)
    similarities = {}
    for user_v in user_nodes:
        if user_u == user_v:
            continue
        if len(list(nx.common_neighbors(G, user_u, user_v))) > 1:
            similarity = similarity_metrics(user_u, user_v)
            similarities[user_v]= similarity
    if len(similarities) > 0:
        #Get users with 10 most similar scores
        similar_users = sorted(similarities, key = similarities.get, reverse=True)[:10]
        user_u_neighbors = list(G.neighbors(user_u))
        recommend_items = set()
        for user_v in similar_users:
            user_v_neighbors = list(G.neighbors(user_v))
            different_neighbors = [node for node in user_v_neighbors if node not in user_u_neighbors]
            for different_node in different_neighbors:
                item_rating = G[user_v][different_node]['weight']
                recommend_items.add((different_node, item_rating))
        #Get 10 most recommended items 
        best_item_nodes = sorted(recommend_items, key = lambda x:x[1], reverse=True)[:10]
        best_item_ids = [item_id[1:] for item_id, score in best_item_nodes] #remove 'i'
    
        #Return the top 10 products as recommendations
        return product_df[product_df['id'].isin(best_item_ids)]
    return None

Next, let's define some metrics to calculate the similarity between 2 users:
- Weighted Pearson Correlation Coefficient(WPCC)
- New Heuristic Similarity Model (NHSM)
- MJaccard similarity
- User Rating Preference similarity (URP)

In [298]:
import random
def baseline(user_u, user_v):
    return random.random()

def jaccard_coef(user_u, user_v):
    return list(nx.jaccard_coefficient(G, [(user_u, user_v)]))[0][2]

from scipy.stats import pearsonr
def weighted_pearson_corr(user_u, user_v, H = 50):
    common_items = list(nx.common_neighbors(G, user_u, user_v))
    user_u_ratings = []
    user_v_ratings = []
    for common_item in common_items:
        user_u_ratings.append(G[user_u][common_item]['weight'])
        user_v_ratings.append(G[user_v][common_item]['weight'])
    pearson_corr = pearsonr(user_u_ratings, user_v_ratings)
    if len(common_items) < H:
        return pearson_corr[0] * len(common_items) / H
    else:
        return pearson_corr[0]

### User ID 2105483 Huỳnh Đức Toàn

In [371]:
customer_id = 2105483

In [372]:
orig_ratings[orig_ratings['customer_id']==customer_id].iloc[0,[0,8]]

customer_id             2105483
customer_name    Huỳnh Đức Toàn
Name: 1705, dtype: object

### Previous bought products

In [384]:
previous_products = product_df[product_df['id'].isin(orig_ratings[orig_ratings['customer_id'] == customer_id]['product_id'])].head()
previous_products = previous_products[['id', 'name', 'description', 'category_name']]
previous_products

,id,name,description,category_name
23,2293833,Combo 2 hộp Nhang Xanh 30cm-500g (Chân Tăm Mộc...,"Thành phần: bột trầm hương, keo bời lời, tăm ...","Nhà Cửa - Đời Sống/Đồ thờ cúng/Hương, nhang"
751,22689010,Combo 3 áo thun trơn nam thời trang Everest mà...,"Chất liệu thun mềm mại co giãn tốt , thoáng má...",Thời trang nam/Áo thun nam
2131,29115771,Quần Jogger Nam Thun Nỉ Ống Bó Có Big Size,Quần jogger nỉ được may bằng chất liệu thun nỉ...,Thời trang nam/Quần nam/Quần jogger nam
6070,4616949,Mũ len kèm khăn,Chất liệu len cao cấp không bông xù. Mặt trong...,Phụ kiện thời trang/Phụ kiện thời trang nam/Nó...
10271,56337264,Xe đạp tập thể dục T-366,Xe đạp tập thể dục T366 MẪU MÃ SẢN PHẨM ĐẸP MẮ...,Thể Thao - Dã Ngoại/Dụng cụ - thiết bị tập thể...


### Recommended products

In [388]:
recommended_products = user_collaborative_recommender(customer_id, jaccard_coef).iloc[4:9]
recommended_products = recommended_products[['id', 'name', 'description', 'category_name']]
recommended_products

,id,name,description,category_name
5054,49047291,Đồng hồ nam FOURRON F1128 santafe watch 2020 L...,Thương hiệu: FOURRON Kiểu Máy: Quartz Kích thư...,Đồng hồ và Trang sức/Đồng hồ nam/Đồng hồ busin...
9933,42901507,Áo thun Nam Nữ unisex ASALA (ux-901),áo thun thoáng mát đa năng unisex nam nữ đều m...,Thời trang nam/Áo thun nam
10354,41067638,Áo giữ nhiệt lót nỉ nam,"Chất liệu vải thun mềm mai, dày dặn ấm áp Thiế...","Thời trang nam/Đồ ngủ, đồ mặc nhà nam/Đồ mặc n..."
14352,26334237,Giày Thể Thao Nam DOL Cao Cổ tăng 7 cm chiều c...,GIÀY BÍ MẬT TĂNG 7CM chiều cao (TẶNG 1 MIẾNG Đ...,Giày - Dép nam/Giày thể thao nam/Giày thể thao...
16041,72673068,Quần Thể Thao Nam 5S (TK004) Cao Cấp Vải Gió D...,Tên sản phẩm: Quần Thể Thao Nam 5S (TK004) Cao...,Thời trang nam/Quần nam/Quần jogger nam
